In [170]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import folium
from folium.plugins import MarkerCluster
from folium.plugins import MousePosition
from folium.features import DivIcon 

import requests

## Mark all launch sites on the map

In [171]:
%%capture
url = "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_geo.csv"
local_path = "./spacex_launch_geo.csv"

response = requests.get(url)

if(response.status_code == 200):
    with open(local_path, 'wb') as file:
        file.write(response.content)
    print(f'Download success, status code {response.status_code}')
else:
    print(f'Error, stastus code {response.status_code}')

path = "spacex_launch_geo.csv"

In [172]:
spacex_df = pd.read_csv(path)
print(spacex_df.shape)
spacex_df.head()

(56, 13)


,Flight Number,Date,Time (UTC),Booster Version,Launch Site,Payload,Payload Mass (kg),Orbit,Customer,Landing Outcome,class,Lat,Long
0,1,2010-06-04,18:45:00,F9 v1.0 B0003,CCAFS LC-40,Dragon Spacecraft Qualification Unit,0.0,LEO,SpaceX,Failure (parachute),0,28.562302,-80.577356
1,2,2010-12-08,15:43:00,F9 v1.0 B0004,CCAFS LC-40,"Dragon demo flight C1, two CubeSats, barrel o...",0.0,LEO (ISS),NASA (COTS) NRO,Failure (parachute),0,28.562302,-80.577356
2,3,2012-05-22,7:44:00,F9 v1.0 B0005,CCAFS LC-40,Dragon demo flight C2+,525.0,LEO (ISS),NASA (COTS),No attempt,0,28.562302,-80.577356
3,4,2012-10-08,0:35:00,F9 v1.0 B0006,CCAFS LC-40,SpaceX CRS-1,500.0,LEO (ISS),NASA (CRS),No attempt,0,28.562302,-80.577356
4,5,2013-03-01,15:10:00,F9 v1.0 B0007,CCAFS LC-40,SpaceX CRS-2,677.0,LEO (ISS),NASA (CRS),No attempt,0,28.562302,-80.577356


In [173]:
#Selecting relevant sub-columnd: 'Launch Site', 'Lat (Ltitude)', 'Long (Longitude)', 'class'

spacex_df = spacex_df[['Launch Site', 'Lat', 'Long', 'class']]
launch_sites_df = spacex_df.groupby(['Launch Site'], as_index=False).first()
launch_sites_df.drop('class', axis=1, inplace=True)
launch_sites_df


,Launch Site,Lat,Long
0,CCAFS LC-40,28.562302,-80.577356
1,CCAFS SLC-40,28.563197,-80.576820
2,KSC LC-39A,28.573255,-80.646895
3,VAFB SLC-4E,34.632834,-120.610745


In [174]:
# Start location is NASA Jhonson Space Center

nasa_coordenate = [29.558160751227824, -95.08931143777262]
site_map = folium.Map(location=nasa_coordenate, zoom_start=10)

In [175]:
LaunchSite_coor = launch_sites_df.to_dict(orient='list')
LaunchSite_coor

{'Launch Site': ['CCAFS LC-40', 'CCAFS SLC-40', 'KSC LC-39A', 'VAFB SLC-4E'],
 'Lat': [28.56230197, 28.56319718, 28.57325457, 34.63283416],
 'Long': [-80.57735648, -80.57682003, -80.64689529, -120.6107455]}

In [176]:
# Add NASA JSC into the map
circle = folium.Circle(nasa_coordenate, radius=1000, color='#d35400', fill=True).add_child(folium.Popup('NASA Johnson Space Center'))
marker = folium.map.Marker(nasa_coordenate, icon=DivIcon(
    icon_size = (20, 20), 
    icon_anchor = (0, 0), 
    html = '<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % 'NASA JSC'
))
site_map.add_child(circle)
site_map.add_child(marker)
###############################################################################################################################################################
# Add Launch sites into the map
for i in range(len(LaunchSite_coor['Lat'])):
    coordenate = [LaunchSite_coor['Lat'][i], LaunchSite_coor['Long'][i]]
    circle = folium.Circle(coordenate, radius=1000, color='#d35400', fill=True).add_child(folium.Popup(LaunchSite_coor['Launch Site'][i]))
    marker = folium.map.Marker(coordenate, icon=DivIcon(
        icon_size = (20, 20), 
        icon_anchor = (0, 0), 
        html = '<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % LaunchSite_coor['Launch Site'][i]
    ))
    site_map.add_child(circle)
    site_map.add_child(marker)

site_map

## Mark the success/failed launches for each site on the map

In [177]:
spacex_df.tail(20)

,Launch Site,Lat,Long,class
36,KSC LC-39A,28.573255,-80.646895,1
37,KSC LC-39A,28.573255,-80.646895,0
38,KSC LC-39A,28.573255,-80.646895,1
39,KSC LC-39A,28.573255,-80.646895,1
40,KSC LC-39A,28.573255,-80.646895,0
41,KSC LC-39A,28.573255,-80.646895,1
42,KSC LC-39A,28.573255,-80.646895,1
43,KSC LC-39A,28.573255,-80.646895,0
44,KSC LC-39A,28.573255,-80.646895,1
45,KSC LC-39A,28.573255,-80.646895,1


In [178]:
marker_color = []
for i in spacex_df['class']:
    if i:
        marker_color.append('green')
    else:
        marker_color.append('red')

spacex_df['Marker Color'] = marker_color
spacex_df

,Launch Site,Lat,Long,class,Marker Color
0,CCAFS LC-40,28.562302,-80.577356,0,red
1,CCAFS LC-40,28.562302,-80.577356,0,red
2,CCAFS LC-40,28.562302,-80.577356,0,red
3,CCAFS LC-40,28.562302,-80.577356,0,red
4,CCAFS LC-40,28.562302,-80.577356,0,red
5,CCAFS LC-40,28.562302,-80.577356,0,red
6,CCAFS LC-40,28.562302,-80.577356,0,red
7,CCAFS LC-40,28.562302,-80.577356,0,red
8,CCAFS LC-40,28.562302,-80.577356,0,red
9,CCAFS LC-40,28.562302,-80.577356,0,red


In [179]:
marker_cluster = MarkerCluster().add_to(site_map)

In [180]:
for index, record in spacex_df.iterrows():
    coordenate = [record['Lat'], record['Long']]
    marker = folium.Marker(location=coordenate, 
                           icon=folium.Icon(color='white', icon_color=record['Marker Color']), 
                           popup=f"Launch Site: {record['Launch Site']} | {'Success' if record['class'] else 'Failure'}"
                           )
    marker_cluster.add_child(marker)
site_map

## Calculate the distance between a Launch site to its proximities

In [181]:
# Add Mouse Position to get the coordinate (lat, long) for a mouse over on the map
formatter = "function(num) {return L.Util.formatNum(num, 5);};"
mouse_position = MousePosition(
    position = 'topright', 
    separator = 'Long: ', 
    empty_string = 'NaN', 
    lng_first=False, 
    num_digits=20, 
    prefix='Lat: ', 
    lat_formatter = formatter, 
    lng_formatter = formatter
)

site_map.add_child(mouse_position)
site_map

In [182]:
from math import cos, sin, sqrt, atan2, radians

def calculate_distance(lat1, lon1, lat2, lon2):
    #Approximate radius of earth in km
    R = 6373.0

    lat1 = radians(lat1)
    lon1 = radians(lon1)
    lat2 = radians(lat2)
    lon2 = radians(lon2)

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat/2)**2 + cos(lat1)*cos(lat2)*sin(dlon/2)**2
    c = 2*atan2(sqrt(a), sqrt(1-a))

    distance = R*c
    return distance

In [183]:
LaunchSite__CCAFS_LC_40 = [28.562302, -80.577356]
LaunchSite__KSC_LC_39A = [28.573255, -80.646895]
LaunchSite__VAFB_SLC_4E = [34.632834, -120.610745]
######
LaunchSite__CCAFS_SLC_40 = [28.563197, -80.576820]
coast_coords = [28.56253, -80.56788]
nasa_railway = [28.57118, -80.58541]
streetSamuelC = [28.56465, -80.57097]
Malbourne = [28.10289, -80.63896]
######

dist_coast = calculate_distance(LaunchSite__CCAFS_SLC_40[0], LaunchSite__CCAFS_SLC_40[1], coast_coords[0], coast_coords[1])
dist_railway = calculate_distance(LaunchSite__CCAFS_SLC_40[0], LaunchSite__CCAFS_SLC_40[1], nasa_railway[0], nasa_railway[1])
dist_street = calculate_distance(LaunchSite__CCAFS_SLC_40[0], LaunchSite__CCAFS_SLC_40[1], streetSamuelC[0], streetSamuelC[1])
dist_city = calculate_distance(LaunchSite__CCAFS_SLC_40[0], LaunchSite__CCAFS_SLC_40[1], Malbourne[0], Malbourne[1])

dist_dict = {'coast': [coast_coords, dist_coast], 'railway': [nasa_railway, dist_railway], 'street': [streetSamuelC, dist_street], 'city': [Malbourne, dist_city]}
dist_dict

{'coast': [[28.56253, -80.56788], 0.8765154321743669],
 'railway': [[28.57118, -80.58541], 1.22172518974357],
 'street': [[28.56465, -80.57097], 0.5939076198938797],
 'city': [[28.10289, -80.63896], 51.560053984250615]}

In [184]:
for i in dist_dict:
    distance_marker = folium.Marker(
        dist_dict[i][0], 
        icon = DivIcon(
            icon_size = (20, 20), 
            icon_anchor = (0, 0), 
            html = '<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % "{:10.2f} KM".format(dist_dict[i][1])
        )
    )
    lines = folium.PolyLine(locations=[LaunchSite__CCAFS_SLC_40, dist_dict[i][0]], weight=1)

    marker_cluster.add_child(distance_marker)
    marker_cluster.add_child(lines)

site_map